In [1]:
import fiona
import os
from ops.ops import load_json, create_exps_paths, load_exp
from tqdm.notebook import tqdm_notebook
from osgeo import gdal
from osgeo import gdalconst
import numpy as np
from tensorflow.keras.utils import to_categorical
from skimage.morphology import area_opening


In [2]:
conf = load_json(os.path.join('conf', 'conf.json'))
img_source = conf['img_source']
batch_size = conf['batch_size']
learning_rate = conf['learning_rate']
n_train_models = conf['n_train_models']
patch_size = conf['patch_size']
n_classes = conf['n_classes']
n_opt_layers = conf['n_opt_layers']
n_sar_layers = conf['n_sar_layers']
class_weights = conf['class_weights']
test_crop = conf['test_crop']
n_imgs = conf['n_imgs']

exp_name = 'rs_sm_opt_pm_nc_3'
exp_path = os.path.join('D:', 'Ferrari', 'exps_7', exp_name)

models_path = os.path.join(exp_path, 'models')
logs_path = os.path.join(exp_path, 'logs')
pred_path = os.path.join(exp_path, 'predicted')
visual_path = os.path.join(exp_path, 'visual')


prep_path = os.path.join('img', 'prepared')

label_path = os.path.join('D:', 'Ferrari', 'source-imgs', 'pa', 'labels')

## Create Geotiff Predictions Probabilities

In [3]:
for test_image in range(n_imgs):
    pred = np.load(os.path.join(pred_path, f'pred_{test_image}.npy'))
    label = np.load(os.path.join(prep_path, f'label_2019.npy'))
    pred_p = pred[:,:,1:2]
    pred_p[label==2] = 0
    in_data = gdal.Open( os.path.join(label_path, f'label_2019.tif'), gdalconst.GA_ReadOnly)
    geo_transform = in_data.GetGeoTransform()
    x_min = geo_transform[0]
    y_max = geo_transform[3]
    x_max = x_min + geo_transform[1] * in_data.RasterXSize
    y_min = y_max + geo_transform[5] * in_data.RasterYSize
    x_res = in_data.RasterXSize
    y_res = in_data.RasterYSize
    crs = in_data.GetSpatialRef()
    proj = in_data.GetProjection()

    output = os.path.join(visual_path, f'pred_prob_{exp_name}_{test_image}.tif')
    target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_Float32)
    
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetSpatialRef(crs)
    target_ds.SetProjection(proj)
    
    band = target_ds.GetRasterBand(1)
    band.WriteArray(pred_p[:,:,0], 0, 0)
    
    band.FlushCache()
    target_ds = None

FileNotFoundError: [Errno 2] No such file or directory: 'D:Ferrari\\exps_7\\rs_sm_opt_pm_nc_3\\predicted\\pred_1.npy'

## Create Geotiff Binary Predictions

In [4]:
for test_image in range(n_imgs):
    pred = np.load(os.path.join(pred_path, f'pred_{test_image}.npy'))
    label = np.load(os.path.join(prep_path, f'label_2019.npy'))
    pred_p = (pred[:,:,1:2]>0.5).astype(np.uint8)
    pred_p[label==2] = 0
    pred_p = area_opening(pred_p.squeeze(), 625)

    in_data =  gdal.Open( os.path.join(label_path, f'label_2019.tif'), gdalconst.GA_ReadOnly)
    geo_transform = in_data.GetGeoTransform()
    x_min = geo_transform[0]
    y_max = geo_transform[3]
    x_max = x_min + geo_transform[1] * in_data.RasterXSize
    y_min = y_max + geo_transform[5] * in_data.RasterYSize
    x_res = in_data.RasterXSize
    y_res = in_data.RasterYSize
    crs = in_data.GetSpatialRef()
    proj = in_data.GetProjection()

    output = os.path.join(visual_path, f'pred_bin_{exp_name}_{test_image}.tif')
    target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_Byte)
    
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetSpatialRef(crs)
    target_ds.SetProjection(proj)
    
    band = target_ds.GetRasterBand(1)
    band.WriteArray(pred_p, 0, 0)
    
    band.FlushCache()
    target_ds = None

FileNotFoundError: [Errno 2] No such file or directory: 'D:Ferrari\\exps_7\\rs_sm_opt_pm_nc_3\\predicted\\pred_1.npy'